In [ ]:
import time, os
from utils.kafka_admin import KafkaAdmin
from utils.generate_data import generate_clickstream_async #args : num_events, latency
from utils.kafka import write_to_kafka_async #args: topic, event_generator, num_events, latency
from utils.spark import start_spark_consumer #args: topic, kafka_bootstrap_servers

# topicname = "clicstream"
# kafka_bootstrap_servers = "localhost:9092"
# number_of_events = 10
# desired_latency = 0.2

schema = """
    user_id INT,
    timestamp STRING,
    event_type STRING,
    page STRING,
    element STRING,
    query STRING,
    product STRING,
    price DOUBLE
"""


In [ ]:


kafka = KafkaAdmin(bootstrap_servers="localhost:9092")
topics = kafka.list_topics()

if len(topics) > 0:
    print("Resetting Kafka topics...\n")
    for topic in topics:
        print(f"Deleting topic {topic}")
        kafka.delete_topic(topic)

    #Wait and poll until topics disappear
    print("Waiting for topics to be fully deleted...")
    for _ in range(10):  # up to ~10 seconds
        time.sleep(1)
        remaining = kafka.list_topics()
        if not any(t in topics for t in remaining):
            break
    else:
        print("Topics still pending deletion, continuing anyway...")


print("\nCreating Kafka topic...")
kafka.create_topic("clickstream")
kafka.close()
print("\nKafka topics reset complete.")


In [ ]:


await write_to_kafka_async("clickstream", generate_clickstream_async(10, 0.2), "localhost:9092")
os.environ["PYSPARK_SUBMIT_ARGS"] = "--conf spark.ui.showConsoleProgress=false pyspark-shell"
start_spark_consumer("clickstream", "localhost:9092",schema)